In [11]:
import pandas as pd
import os

In [19]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
from torchvision import transforms

# Custom Dataset Generator with image and LiDAR preprocessing
class ImitationLearningDataset(Dataset):
    def __init__(self, control_data, lidar_data, camera_data, image_size=(480,640)):
        self.control_data = control_data
        self.lidar_data = lidar_data
        self.camera_data = camera_data

        # Define preprocessing transformations for the images
        self.image_preprocess = transforms.Compose([
            transforms.Resize(image_size),         # Resize image
            transforms.ToTensor(),                 # Convert image to tensor
            transforms.Normalize(                  # Normalize with ImageNet mean and std
                mean=[0.485, 0.456, 0.406],        # Mean values for R, G, B channels
                std=[0.229, 0.224, 0.225]          # Standard deviations for R, G, B channels
            ),
        ])

    def __len__(self):
        return len(self.control_data)

    def preprocess_lidar(self, lidar_path, max_points=2048):
        # Load the LiDAR point cloud data
        point_cloud = np.load(lidar_path)
        # Transpose the dimensions of the point cloud
        point_cloud = np.transpose(point_cloud, (1, 0))  # Shape: (3, N)
        N = point_cloud.shape[1]
    
        if N > max_points:
            # Randomly sample max_points
            indices = np.random.choice(N, max_points, replace=False)
            point_cloud = point_cloud[:, indices]
        elif N < max_points:
            # Pad with zeros
            padding = np.zeros((3, max_points - N))
            point_cloud = np.concatenate((point_cloud, padding), axis=1)
    
        # Convert to a PyTorch tensor
        lidar_tensor = torch.tensor(point_cloud, dtype=torch.float32)
        return lidar_tensor


    def __getitem__(self, idx):
        # Control data as the base
        control = self.control_data.iloc[idx]
        control_timestamp = control['timestamp']

        # Find the closest previous timestamp in LiDAR and camera data
        lidar_past = self.lidar_data[self.lidar_data['timestamp'] <= control_timestamp]
        camera_past = self.camera_data[self.camera_data['timestamp'] <= control_timestamp]

        # Handle case where no past timestamps are found (default to the first entry)
        if lidar_past.empty:
            closest_lidar_idx = 0
        else:
            closest_lidar_idx = (control_timestamp - lidar_past['timestamp']).idxmin()

        if camera_past.empty:
            closest_camera_idx = 0
        else:
            closest_camera_idx = (control_timestamp - camera_past['timestamp']).idxmin()

        # Load the LiDAR and Image data
        lidar_path = self.lidar_data.iloc[closest_lidar_idx]['lidar_file']
        image_path = self.camera_data.iloc[closest_camera_idx]['image_file']

        # Preprocess image
        image = Image.open(image_path).convert('RGB')  # Convert to RGB if needed
        image_tensor = self.image_preprocess(image)    # Apply the preprocessing pipeline

        # Preprocess LiDAR
        lidar_tensor = self.preprocess_lidar(lidar_path)

        # Control targets (throttle, brake, steering)
        target = torch.tensor([control['throttle'], control['brake'], control['steering']], dtype=torch.float32)

        return image_tensor, lidar_tensor, target  # LiDAR already has the batch dimension added


In [20]:
control_data = pd.read_csv('carla_data/control_data.csv')
lidar_data = pd.read_csv('carla_data/lidar_data.csv')
camera_data = pd.read_csv('carla_data/camera_data.csv')


In [21]:
from actor import Actor

In [22]:
from tqdm import tqdm 

In [23]:
pretrained_weights = torch.load(
                        "./pointnet_torch/log/sem_seg/pointnet2_sem_seg/checkpoints/best_model.pth"
)

pretrained_weights_state = pretrained_weights['model_state_dict']
intensity_weights = torch.mean(pretrained_weights_state['sa1.mlp_convs.0.weight'][:, 3:-3,:,:], dim=1, keepdim=True)
pretrained_weights_state['sa1.mlp_convs.0.weight'].shape
partial_weights = torch.cat([pretrained_weights_state['sa1.mlp_convs.0.weight'][:, :3,:,:],
                            pretrained_weights_state['sa1.mlp_convs.0.weight'][:, -3:,:,:]], dim=1)
pretrained_weights_state['sa1.mlp_convs.0.weight'] = partial_weights
partial_weights_state = {k:v for k,v in  pretrained_weights_state.items() if k.startswith('sa')}

C:\Users\Aufa\AppData\Local\Temp\ipykernel_416\3650187859.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_weights = torch.load(


In [24]:
# Initialize Dataset and DataLoader
dataset = ImitationLearningDataset(control_data, lidar_data, camera_data)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# Initialize model, loss, and optimizer
actor = Actor(
    image_shape = (480,640),
    pointnet_weights = partial_weights_state
)
criterion = torch.nn.MSELoss()  # Mean Squared Error for regression
optimizer = torch.optim.Adam(actor.parameters(), lr=1e-4)

C:\Users\Aufa\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Aufa\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
# Training loop
def train_model(model, dataloader, criterion, optimizer, num_epochs=10, save_dir='models/'):
    model.train()
    os.makedirs(save_dir, exist_ok=True)
    for epoch in range(num_epochs):
        print(f"\nEpoch [{epoch+1}/{num_epochs}]")  # Print new line and epoch info
        
        running_loss = 0.0

        # Initialize tqdm progress bar for the current epoch's batches
        for images, lidars, targets in tqdm(dataloader, desc=f"Batch Progress", leave=True):
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass through the model
            predicted_actions = model(images, lidars)

            # Compute loss
            loss = criterion(predicted_actions, targets)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Accumulate running loss for tracking
            running_loss += loss.item()

        # Print the average loss for the epoch
        avg_loss = running_loss / len(dataloader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

        # Save the model at the end of each epoch
        model_save_path = f'{save_dir}model_epoch_{epoch+1}.pth'
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
        }, model_save_path)
        print(f"Model saved at: {model_save_path}")

    print("Training complete.")


In [10]:

# Start training
train_model(actor, dataloader, criterion, optimizer, num_epochs=20)


Epoch [1/20]


Batch Progress: 100%|██████████████████████████████████████████████████████████| 10/10 [03:49<00:00, 22.90s/it]


Epoch [1/20], Loss: 0.1024
Model saved at: models/model_epoch_1.pth

Epoch [2/20]


Batch Progress: 100%|██████████████████████████████████████████████████████████| 10/10 [03:51<00:00, 23.14s/it]


Epoch [2/20], Loss: 0.0784
Model saved at: models/model_epoch_2.pth

Epoch [3/20]


...

Epoch [18/20], Loss: 0.0578
Model saved at: models/model_epoch_18.pth

Epoch [19/20]


Batch Progress: 100%|██████████████████████████████████████████████████████████| 10/10 [03:44<00:00, 22.46s/it]


Epoch [19/20], Loss: 0.0573
Model saved at: models/model_epoch_19.pth

Epoch [20/20]


Batch Progress: 100%|██████████████████████████████████████████████████████████| 10/10 [03:46<00:00, 22.60s/it]


Epoch [20/20], Loss: 0.0582
Model saved at: models/model_epoch_20.pth
Training complete.


In [29]:
save_dir = 'checkpoint/'
model_save_path = f'{save_dir}model_epoch_{1}.pth'

tensor(0.0245, grad_fn=<MseLossBackward0>)

In [35]:
model_save_path = 'models/imitation_model.pth'

mp = torch.load(model_save_path)

C:\Users\Aufa\AppData\Local\Temp\ipykernel_3488\1481851779.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mp = torch.load(model_save_path)


In [36]:
mp_state = mp['model_state_dict']

In [10]:
actor = Actor()
actor.load_state_dict(mp_state)

In [27]:
# Test saved Model
dataset = ImitationLearningDataset(control_data, lidar_data, camera_data)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)


for images, lidars, targets in  dataloader:
    images = images[0]
    lidars = lidars[0]
    targets = targets[0]
    predicted_actions = actor(images,lidars)
    loss = criterion(predicted_actions, targets)
    break

C:\Users\Aufa\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([26, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
predicted_actions